In [ ]:
import os
import sys
import glob
import numpy as np
from scipy.fft import fft, fftfreq
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})
OU_tau = 20e-3

In [ ]:
expt_name = 'default_const_P_loads'
# expt_name = 'G02_double_H_const_P_loads'
folder = os.path.join('..','..','modal_analysis','IEEE39',expt_name)
AC_data_file = os.path.join(folder,'IEEE39_AC_TF_-6.0_2.0_1000.npz')
AC_data = np.load(AC_data_file, allow_pickle=True)
dB = AC_data['dB'].item()

In [ ]:
tran_data_file = os.path.join(folder,'IEEE39_tran.npz')
tran_blob = np.load(tran_data_file, allow_pickle=True)
tran_time = tran_blob['time']
tran_data = tran_blob['data'].item()
tran_speed = tran_data['gen']['s:xspeed']

In [ ]:
tran_M = tran_blob['momentum'].item()
AC_M = AC_data['Mtot'].item()
M = tran_M

In [ ]:
dur = 900 # [s]
dt = tran_time[1] - tran_time[0]
n_samples = int(dur/dt)
tran_freq = fftfreq(n_samples, dt)[:n_samples//2]
tran_speedf = np.zeros((tran_freq.size, tran_speed.shape[1]))
for j in range(tran_speed.shape[1]):
    n_trials = tran_speed.shape[0] // n_samples
    X = np.reshape(tran_speed[:n_trials*n_samples,j], (n_trials, n_samples))
    tmp = fft(X-1)
    tran_speedf[:,j] = np.mean(2.0 / n_samples * np.abs(tmp[:,:n_samples//2]), axis=0)

In [ ]:
device_names = tran_blob['device_names'].item()['gen']
if len(device_names) == 10:
    rows,cols = 2,5
elif len(device_names) == 18:
    rows,cols = 3,6
w,h = 3,2.5
fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h), sharex=True, sharey=True)
ticks = np.logspace(-4, 2, 7)
F0 = 0.1
col = 'kr'
cmap = plt.get_cmap('tab10', len(AC_data))
lw = 1
names = tran_blob['device_names'].item()['gen']
AC_freq = AC_data['F']
for k,name in enumerate(device_names[:rows*cols]):
    i,j = k//cols, k%cols
    try:
        idx = names.index(name)
        jdx = np.abs(tran_freq-F0).argmin()
        y = dB*np.log10(tran_speedf[:,idx])
        ax[i,j].semilogx(tran_freq[1:], y[1:]-y[jdx], 'k', lw=lw, label='M = {:.0f} MJs'.format(M))
        idx = np.where(AC_data['SM_names'] == name)[0][0]
        AC_freq = AC_data['F']
        jdx = np.abs(AC_data['F']-F0).argmin()
        y = AC_data['mag'][:,idx]
        ax[i,j].semilogx(AC_freq, y-y[jdx], 'r', lw=1)
        ax[i,j].plot(1/OU_tau/(2*np.pi)+np.zeros(2), [-80,20], ':', color=[0,.8,.4], lw=2)
        ax[i,j].set_title(name.split('___')[0], fontsize=8)
        ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
        ax[i,j].xaxis.set_minor_locator(NullLocator())
        ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
        ax[i,j].set_xlim(ticks[[0,-1]])
    except:
        print(f'Device name {name} missing')
ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel(f'[dB{dB}]')
sns.despine()
fig.tight_layout()
plt.savefig(os.path.join(folder, 'spectra_comparison.pdf'))